<a href="https://colab.research.google.com/github/cagBRT/Clustering-Intro/blob/master/Clustering_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook you create a clustering pipeline using a real-world dataset.

# **Import Libraries**

In [ ]:
import tarfile
import urllib

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

These data contain gene expression values from a manuscript authored by The Cancer Genome Atlas (TCGA) Pan-Cancer analysis project investigators.

There are 881 samples (rows) representing five distinct cancer subtypes. Each sample has gene expression values for 20,531 genes (columns). The dataset is available from the UC Irvine Machine Learning Repository.

In [ ]:
uci_tcga_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00401/"
archive_name = "TCGA-PANCAN-HiSeq-801x20531.tar.gz"

# build the url
full_download_url = urllib.parse.urljoin(uci_tcga_url, archive_name)

# download the file
r = urllib.request.urlretrieve (full_download_url, archive_name)

# extract the data from the archive
tar = tarfile.open(archive_name, "r:gz")
tar.extractall()
tar.close()

In [ ]:
datafile = "TCGA-PANCAN-HiSeq-801x20531/data.csv"
labels_file = "TCGA-PANCAN-HiSeq-801x20531/labels.csv"

data = np.genfromtxt(
    datafile,
    delimiter=",",
    usecols=range(1, 20532),
    skip_header=1
)

true_label_names = np.genfromtxt(
    labels_file,
    delimiter=",",
    usecols=(1,),
    skip_header=1,
    dtype=str
)

The data has 801 rows and 20,531 columns.

In [ ]:
data.shape

Print the first 5 rows and the first three columns of tge dataset

In [ ]:
data[:5, :3]

There are five classes of cancer in the dataset. <br>
These are the labels (predictions) that will be made with the clustering model.

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
true_labels = label_encoder.fit_transform(true_label_names)

The labels are strings containing abbreviations of cancer types:<br>

>BRCA: Breast invasive carcinoma<br>
COAD: Colon adenocarcinoma<br>
KIRC: Kidney renal clear cell carcinoma<br>
LUAD: Lung adenocarcinoma<br>
PRAD: Prostate adenocarcinoma<br>


In [ ]:
label_encoder.classes_

Print a sample of the true_label_names

In [ ]:
true_label_names[:5]

Print a sample of the converted label names

In [ ]:
true_labels[:5]

The number of clusters is equal to the number of classes (or labels)

In [ ]:
n_clusters = len(label_encoder.classes_)

# **Data PreProcessing Pipelines**

Many times, the data needs to be processed before it can be used in the machine learning model. <br>
Creating and using a data pipeline means the data can be consistently treated during development and production release. 

As the number of features increases, the feature space becomes sparse. This sparsity makes it difficult for algorithms to find data objects near one another in higher-dimensional space. Since the gene expression dataset has over 20,000 features, it qualifies as a great candidate for dimensionality reduction.

**Principal Component Analysis** (PCA) is one of many dimensionality reduction techniques. PCA transforms the input data by projecting it into a lower number of dimensions called components. The components capture the variability of the input data through a linear combination of the input data’s features.<br><br>
*If you are interested in learning more about PCA, BRT offers a 1/2 day class on it.* 

**Create the preprocessor pipeline for the data**<br>
Scale the data<br>
Perform PCA<br>

In [ ]:
preprocessor = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        ("pca", PCA(n_components=2, random_state=42)),
    ]
)

**Create the pipeline to perform clustering**

In [ ]:
#k++ == selects initial cluster centers for k-mean  
    #clustering in a smart way to speed up convergence. See section  
#n_init == Number of time the k-means algorithm will be run with different  
    #centroid seeds. The final results will be the best output of  
#max_iter== You’ll increase the number of iterations per initialization
    #to ensure that k-means will converge.

clusterer = Pipeline(
   [
       (
           "kmeans",
           KMeans(
               n_clusters=n_clusters, #for this data, 5 clusters
               init="k-means++", 
               n_init=50,
               max_iter=500,
               random_state=42,
           ),
       ),
   ]
)

Combine the two pipelines to form an end-to-end k-means clustering pipeline

In [ ]:
pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("clusterer", clusterer)
    ]
)

**Train the model**

In [ ]:
pipe.fit(data)

**Evaluate the performance of the model**

The scale for each of these clustering performance metrics ranges from -1 to 1. <br>

A silhouette coefficient of 0 indicates that clusters are significantly overlapping one another,<br>

 A silhouette coefficient of 1 indicates clusters are well-separated.<br>

An ARI score of 0 indicates that cluster labels are randomly assigned,<br>

An ARI score of 1 means that the true labels and predicted labels form identical clusters.

In [ ]:
preprocessed_data = pipe["preprocessor"].transform(data)

In [ ]:
predicted_labels = pipe["clusterer"]["kmeans"].labels_

In [ ]:
silhouette_score(preprocessed_data, predicted_labels)

In [ ]:
adjusted_rand_score(true_labels, predicted_labels)

In [ ]:
pcadf = pd.DataFrame(
    pipe["preprocessor"].transform(data),
    columns=["component_1", "component_2"],
)

pcadf["predicted_cluster"] = pipe["clusterer"]["kmeans"].labels_
pcadf["true_label"] = label_encoder.inverse_transform(true_labels)

The visual representation of the clusters confirms the results of the two clustering evaluation metrics. <br>
The performance of the pipeline is pretty good. The clusters only slightly overlapped, and cluster assignments were much better than random.

In [ ]:
plt.style.use("fivethirtyeight")
plt.figure(figsize=(8, 8))

scat = sns.scatterplot(
    "component_1",
    "component_2",
    s=50,
    data=pcadf,
    hue="predicted_cluster",
    style="true_label",
    palette="Set2",
)

scat.set_title(
    "Clustering results from TCGA Pan-Cancer\nGene Expression Data"
)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

plt.show()

# **Tuning the Pipeline**
The process of parameter tuning consists of sequentially altering one of the input values of the algorithm’s parameters and recording the results.<br>
Setting the PCA parameter n_components=2, you squished all the features into two components, or dimensions. This value was convenient for visualization on a two-dimensional plot. <br>
Using only two components means that the PCA step won’t capture all of the explained variance of the input data.

In [ ]:
# Empty lists to hold evaluation metrics
silhouette_scores = []
ari_scores = []
for n in range(2, 11):
    # This set the number of components for pca,
    # but leaves other steps unchanged
    pipe["preprocessor"]["pca"].n_components = n
    pipe.fit(data)

    silhouette_coef = silhouette_score(
        pipe["preprocessor"].transform(data),
        pipe["clusterer"]["kmeans"].labels_,
    )
    ari = adjusted_rand_score(
        true_labels,
        pipe["clusterer"]["kmeans"].labels_,
    )

    # Add metrics to their lists
    silhouette_scores.append(silhouette_coef)
    ari_scores.append(ari)

In [ ]:
plt.style.use("fivethirtyeight")
plt.figure(figsize=(8, 8))
plt.plot(
    range(2, 11),
    silhouette_scores,
    c="#008fd5",
    label="Silhouette Coefficient",
)
plt.plot(range(2, 11), ari_scores, c="#fc4f30", label="ARI")

plt.xlabel("n_components")
plt.legend()
plt.title("Clustering Performance as a Function of n_components")
plt.tight_layout()
plt.show()

The silhouette coefficient decreases linearly. The silhouette coefficient depends on the distance between points, so as the number of dimensions increases, the sparsity increases.<br>


The ARI improves significantly as you add components. It appears to start tapering off after n_components=7, so that would be the value to use for presenting the best clustering results from this pipeline.

# **Change the pipeline to the new value**


In [ ]:
#change the PCA components to a recommended value
preprocessor = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        ("pca", PCA(n_components=#, random_state=42)),
    ]
)

In [ ]:
pipe.fit(data)

As with most machine learning decisions, you must balance optimizing clustering evaluation metrics with the goal of the clustering task. <br>
When cluster labels are available, ARI is a reasonable choice. <br>
ARI quantifies how accurately your pipeline was able to reassign the cluster labels.

In [ ]:
preprocessed_data = pipe["preprocessor"].transform(data)
predicted_labels = pipe["clusterer"]["kmeans"].labels_
silhouette_score(preprocessed_data, predicted_labels)

The silhouette coefficient is a good choice for exploratory clustering because it helps to identify subclusters.<br>

These subclusters warrant additional investigation, which can lead to new and important insights.

In [ ]:
adjusted_rand_score(true_labels, predicted_labels)